## FROC

In [36]:
import os
import pandas as pd


In [42]:
gt_file = rf'pirads_gt.xlsx'
reader_file = rf'pirads_reader.xlsx'

In [43]:
gt_df = pd.read_excel(gt_file)
reader_df = pd.read_excel(reader_file)
case_list = ['P' + (3-len(str(x))) * '0' + str(x) for x in range(1,181)]

In [45]:
reader_df

,Round,Reader,PID,Lesion_Idx,Phys_Z,Phys_Y,Phys_X,Score
0,1,KQQ,P001,1.0,-19.9611,18.6213,-1.7860,3.0
1,1,KQQ,P002,1.0,-24.6489,7.9376,23.5730,5.0
2,1,KQQ,P003,1.0,4.4922,-2.6093,-29.1480,5.0
3,1,KQQ,P003,2.0,-13.0859,-4.9531,-33.6480,5.0
4,1,KQQ,P003,3.0,5.6641,20.4376,-33.6480,5.0
...,...,...,...,...,...,...,...,...
3122,2,WZ,P176,NaN,NaN,NaN,NaN,NaN
3123,2,WZ,P177,NaN,NaN,NaN,NaN,NaN
3124,2,WZ,P178,NaN,NaN,NaN,NaN,NaN
3125,2,WZ,P179,NaN,NaN,NaN,NaN,NaN


In [56]:
threshold = 3
roundx = 1
readers = ['KQQ']
for case in case_list:
    gt_lesions = gt_df[gt_df.PID == case]
    for reader in readers:
        reader_lesions = reader_df.loc[(reader_df.PID == case) & (reader_df.Reader == reader) & (reader_df.Round == roundx)]
        

TypeError: Cannot perform 'rand_' with a dtyped [object] array and scalar of type [bool]

In [70]:
test = reader_df.loc[(reader_df.PID == 'P003') & (reader_df.Reader == 'KQQ') & (reader_df.Round == 1)]
test.Phys_Z

2     4.4922
3   -13.0859
4     5.6641
5   -17.3828
6   -14.6484
7    -5.2734
8     2.9297
Name: Phys_Z, dtype: float64

## Kappa & Accuracy

In [94]:
from sklearn.metrics import cohen_kappa_score
import numpy as np
from scipy.spatial.distance import pdist

def valid_lesion_detection(pred_loc, gt_loc, distance_threshold=10):
    pred_pt = np.array(pred_loc)
    gt_pt = np.array(gt_loc)
    dist_matrix = np.vstack((pred_pt, gt_pt))
    distance = pdist(dist_matrix, 'euclidean')
    return True if distance <= distance_threshold else False

def lesion_kappa(pred_results, gt_results, case_list, roundx, reader = 'All', weights_method ='linear',threshold=3):
    gt_lesion_pirads = []
    pred_lesion_pirads = []
    corrected_count = 0
    for case in case_list:
        if reader == 'All':
            cur_pred_df = pred_results[(pred_results.PID == case) & (pred_results.Round == roundx)]    
        else:
            cur_pred_df = pred_results[(pred_results.PID == case) & (pred_results.Round == roundx) & (pred_results.Reader == reader)]
        cur_gt_df = gt_results[(gt_results.PID == case)]
        
        #gt_lesions_info = gt_results[gt_key].lesions_info
        #gt_selected_lesion = [x for x in gt_lesions_info if x.pirads_comb >= threshold]
        gt_selected_lesion = cur_gt_df[cur_gt_df.Score>=threshold]
        pred_selected_lesion = cur_pred_df[cur_pred_df.Score>=threshold]
        gt_lesion_count = len(gt_selected_lesion)
        pred_lesion_count = len(pred_selected_lesion)
        if gt_lesion_count == 0:
            continue
        elif pred_lesion_count == 0:
            continue
        else:
            gt_lesion_status = [0] * gt_lesion_count   
            pred_lesion_status = [0] * pred_lesion_count       
            for i in range(gt_lesion_count):
                cur_gt_lesion_loc = gt_selected_lesion.iloc[i].values[2:-2]
                cur_gt_lesion_pirads = gt_selected_lesion.iloc[i].values[-2]
                for j in range(pred_lesion_count):
                    if pred_lesion_status[j] == 1:
                        continue
                    cur_pred_lesion_loc = pred_selected_lesion.iloc[j].values[4:-1]
                    cur_pred_lesion_pirads = pred_selected_lesion.iloc[j].values[-1]
                    if valid_lesion_detection(cur_gt_lesion_loc, cur_pred_lesion_loc):
                        gt_lesion_pirads.append(cur_gt_lesion_pirads)
                        pred_lesion_pirads.append(cur_pred_lesion_pirads)
                        if cur_gt_lesion_pirads == cur_pred_lesion_pirads:
                            corrected_count += 1
                        gt_lesion_status[i] = 1
                        pred_lesion_status[j] = 1
                        continue
                if gt_lesion_status[i] == 1:
                    continue
           
    kappa_score = cohen_kappa_score(gt_lesion_pirads, pred_lesion_pirads, weights=weights_method)
    accuracy = corrected_count / len(pred_lesion_pirads)                
    return round(kappa_score,4), round(accuracy,4)

In [95]:
reader_lesion = r'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\codes\Prostate_MR\Pilot_Study\pirads_reader.xlsx'
gt_lesion =r'D:\UserData\z004b0je\OneDrive - Siemens Healthineers\codes\Prostate_MR\Pilot_Study\pirads_gt.xlsx'

In [96]:
import pandas as pd

In [97]:
case_list = ['P' + (3-len(str(x)))*'0' + str(x) for x in range(1,181)]
reader_df = pd.read_excel(reader_lesion)
gt_df = pd.read_excel(gt_lesion)
readers = ['KQQ','LF','WHC','YJY','ZHY','WL','WZ','All']
pilot_lesion_results = dict()
cols = ['Round', 'Reader', 'Kappa', 'Accuracy']
for col in cols:
    pilot_lesion_results.setdefault(col, [])
for Round in [1,2]:
    for Reader in readers:
        Kappa, Accuracy = lesion_kappa(reader_df, gt_df, case_list, Round, Reader, weights_method ='linear',threshold=3)
        for col in cols:
            pilot_lesion_results[col].append(eval(col))

In [99]:
pilot_lesion_df = pd.DataFrame(pilot_lesion_results)
pilot_lesion_df.to_excel('lesion_results.xlsx', index=False)

In [ ]:
0.5523255813953488 0.7272727272727273

In [54]:
pred_lesion_pirads

NameError: name 'pred_lesion_pirads' is not defined

In [34]:
test

,Round,Reader,PID,Lesion_Idx,Phys_Z,Phys_Y,Phys_X,Score
1495,2,KQQ,P010,1.0,-14.5715,-8.0006,0.978,5.0
1496,2,KQQ,P010,2.0,-32.1496,2.5463,-3.522,4.0
1497,2,KQQ,P010,3.0,-0.8996,10.7494,0.978,5.0
1498,2,KQQ,P010,4.0,-31.3684,-7.2193,9.978,3.0


In [36]:
test[test.Score<5]

,Round,Reader,PID,Lesion_Idx,Phys_Z,Phys_Y,Phys_X,Score
1496,2,KQQ,P010,2.0,-32.1496,2.5463,-3.522,4.0
1498,2,KQQ,P010,4.0,-31.3684,-7.2193,9.978,3.0


In [39]:
test.iloc[0].values[4:-1]

array([-14.5715, -8.0006, 0.978], dtype=object)

In [77]:
gt_df

,PID,Lesion_Idx,Phys_Z,Phys_Y,Phys_X,Score,Contri
0,P001,1.0,-18.7892,37.1760,2.7140,3.0,2.0
1,P002,1.0,-13.3208,5.5938,32.5730,4.0,2.0
2,P003,1.0,5.2734,-2.0885,-33.6480,5.0,3.0
3,P004,1.0,-17.5005,7.9562,18.5430,4.0,3.0
4,P005,1.0,4.1011,-6.0040,20.7929,5.0,2.0
...,...,...,...,...,...,...,...
185,P176,NaN,NaN,NaN,NaN,NaN,NaN
186,P177,NaN,NaN,NaN,NaN,NaN,NaN
187,P178,NaN,NaN,NaN,NaN,NaN,NaN
188,P179,NaN,NaN,NaN,NaN,NaN,NaN
